In [1]:
import torch.optim as optim
import torch.nn.functional as F
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import os
import torchvision.transforms as transforms
import torch.utils.data as data
from torchvision import datasets 
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from torch.utils.data import Dataset
from PIL import Image
from glob import glob
from os.path import basename

from torchvision.models import mobilenet_v2

In [2]:
train_path = '/StudentData/hw2_data/train_images/train'
test_path = '/StudentData/hw2_data/test_images/test'

In [3]:
# Hyper Parameters
num_epochs = 10
batch_size = 64
learning_rate = 0.001

In [4]:
# Create Data Loader for Image


class ImageFolderWithName(Dataset):
    def __init__(self, root, transform):
        self.transform = transform
        self.images_paths = glob(root + '/*.jpg')
        self.labels = torch.LongTensor([int(basename(image_path).split('_')[1].split('.')[0]) for image_path in self.images_paths])

    def __len__(self):
        return len(self.images_paths)

    def __getitem__(self, idx):
        image_path = self.images_paths[idx]
        image = self.transform(Image.open(image_path))
        label = self.labels[idx]
        return (image, label, basename(image_path))

In [5]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomResizedCrop(224),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225))
])


transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225))  
])    

image_datasets_train_1 = ImageFolderWithName(train_path, transform=transform_train)
image_datasets_train_2 = ImageFolderWithName(train_path, transform=transform_test)
image_datasets_test = ImageFolderWithName(test_path, transform=transform_test)

train_loader_1 = data.DataLoader(image_datasets_train_1,
                                batch_size=batch_size,
                                shuffle=True) 

train_loader_2 = data.DataLoader(image_datasets_train_2,
                                batch_size=batch_size,
                                shuffle=False) 

test_loader = data.DataLoader(image_datasets_test,
                                batch_size=batch_size,
                                shuffle=False)  

In [6]:
def to_gpu(x):
    return x.cuda() if torch.cuda.is_available() else x

In [7]:
def evaluation_acc(labels, outputs):
    return accuracy_score(labels, outputs)

def evaluation_ROC(labels, outputs):
    return roc_auc_score(labels, outputs)

def evaluation_F1(labels, outputs):
    return f1_score(labels, outputs, average='binary')

In [8]:
net = mobilenet_v2()

In [9]:
net.parameters

<bound method Module.parameters of MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96,

In [10]:
net.classifier = nn.Sequential(
  nn.Dropout(p=0.2, inplace=False),
  nn.Linear(in_features=1280, out_features=2, bias=True)
)

In [11]:
Net = net
Net = to_gpu(Net)

criterion = to_gpu(nn.CrossEntropyLoss())
optimizer = torch.optim.Adam(Net.parameters(), lr=learning_rate)

print('number of parameters: ', sum(param.numel() for param in Net.parameters()))
print(f'Num of trainable parameters : {sum(p.numel() for p in Net.parameters() if p.requires_grad)}')

number of parameters:  2226434
Num of trainable parameters : 2226434


In [12]:
loss_test_arr = []
loss_train_arr = []
acc_train_arr = []
acc_test_arr = []
ROC_train_arr = []
ROC_test_arr = []
F1_train_arr = []
F1_test_arr = []
names_arr = []
for epoch in range(num_epochs):
    batchs_loss_train = []
    batchs_loss_test = []
    batchs_acc_train = []
    batchs_acc_test = []
    labels_train_arr = []
    labels_test_arr = []
    predicted_train = []
    predicted_test = []
    Net.train()
    for i, (images, labels, names) in enumerate(train_loader_1):
        images = to_gpu(images)
        labels = to_gpu(labels)
        optimizer.zero_grad()
        outputs = Net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if (i+1) % 200 == 0:
            print ('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, i+1, len(image_datasets_train_1)//batch_size, loss.item()))
    
    Net.eval()
    for images_test, labels_test, names_test in train_loader_2:
        images_test = to_gpu(images_test)
        labels_test = to_gpu(labels_test)
        outputs_test = Net(images_test)
        foo, predicted = torch.max(outputs_test.data, 1)
        predicted_train = np.append(predicted_train, predicted.cpu().numpy())
        labels_train_arr = np.append(labels_train_arr, labels_test.cpu().numpy())
        loss_test = criterion(outputs_test, labels_test)
        batchs_loss_train.append((loss.item())*len(images_test))
    
    loss_train_arr.append(sum(batchs_loss_train) / len(image_datasets_train_2))
    acc_train_arr.append(evaluation_acc(labels_train_arr, predicted_train))
    ROC_train_arr.append(evaluation_ROC(labels_train_arr, predicted_train))
    F1_train_arr.append(evaluation_F1(labels_train_arr, predicted_train))
    print("train acc: ", acc_train_arr[-1])
    
    for images_test, labels_test, names_test in test_loader:
        images_test = to_gpu(images_test)
        labels_test = to_gpu(labels_test)
        outputs_test = Net(images_test)
        foo, predicted = torch.max(outputs_test.data, 1)
        predicted_test = np.append(predicted_test, predicted.cpu().numpy())
        labels_test_arr = np.append(labels_test_arr, labels_test.cpu().numpy())
        names_arr.append(names_test)
        loss_test = criterion(outputs_test, labels_test)
        batchs_loss_test.append((loss_test.item())*len(images_test))
    
    loss_test_arr.append(sum(batchs_loss_test) / len(image_datasets_test))
    acc_test_arr.append(evaluation_acc(labels_test_arr, predicted_test))
    ROC_test_arr.append(evaluation_ROC(labels_test_arr, predicted_test))
    F1_test_arr.append(evaluation_F1(labels_test_arr, predicted_test))
    print("test acc: ", acc_test_arr[-1])

Epoch: [1/10], Step: [200/285], Loss: 0.2989
train acc:  0.8996659181773372
test acc:  0.8982911600394348
Epoch: [2/10], Step: [200/285], Loss: 0.2934
train acc:  0.9385508516348102
test acc:  0.9441340782122905
Epoch: [3/10], Step: [200/285], Loss: 0.2253
train acc:  0.9535571498986801
test acc:  0.9544857048964838
Epoch: [4/10], Step: [200/285], Loss: 0.2333
train acc:  0.9489019113861658
test acc:  0.9508708511337496
Epoch: [5/10], Step: [200/285], Loss: 0.1917
train acc:  0.9545977326250068
test acc:  0.9581005586592178
Epoch: [6/10], Step: [200/285], Loss: 0.2829
train acc:  0.9671394928528397
test acc:  0.9682878738087414
Epoch: [7/10], Step: [200/285], Loss: 0.1257
train acc:  0.9690563557697573
test acc:  0.9696023660860993
Epoch: [8/10], Step: [200/285], Loss: 0.3644
train acc:  0.9640725121857714
test acc:  0.9643443969766677
Epoch: [9/10], Step: [200/285], Loss: 0.1842
train acc:  0.9728900816035927
test acc:  0.9699309891554387
Epoch: [10/10], Step: [200/285], Loss: 0.1597


In [13]:
import pickle
with open("loss_test5.txt", "wb") as fp:   #Pickling
    pickle.dump(loss_test_arr, fp)
with open("loss_train5.txt", "wb") as fp:   #Pickling
    pickle.dump(loss_train_arr, fp)
with open("acc_train5.txt", "wb") as fp:     
    pickle.dump(acc_train_arr, fp)
with open("acc_test5.txt", "wb") as fp: 
    pickle.dump(acc_test_arr, fp)
with open("ROC_train5.txt", "wb") as fp: 
    pickle.dump(ROC_train_arr, fp)
with open("ROC_test5.txt", "wb") as fp: 
    pickle.dump(ROC_test_arr, fp)
with open("F1_train5.txt", "wb") as fp: 
    pickle.dump(F1_train_arr, fp)
with open("F1_test5.txt", "wb") as fp: 
    pickle.dump(F1_test_arr, fp)

In [15]:
torch.save(Net.state_dict(), "net_model_1.pt")